# Setup

Before we start developing with Elasticsearch, let's make sure we can 

- connect to the cluster
- index some data
- and perform a query on the index

In [ ]:
from elasticsearch import Elasticsearch

# Test Connection 
client = Elasticsearch(hosts="http://localhost:9200", basic_auth=('elastic', 'changeme'))

assert client.cluster.health()['status'] == 'green', 'Cluster not healthy'

The connection should be established and the cluster health green - which means we're good to go!

In [ ]:
# Create an index
response = client.indices.create(index='my-first-index')

assert response['acknowledged'], 'Index could not be created'
assert client.indices.exists(index='my-first-index')

In [ ]:
# Add a sample document
client.index(index='my-first-index',id=23,document={
    'message': 'Michael Jeffrey Jordan (born February 17, 1963), also known by his initials MJ, is an American businessman and former professional basketball player. He played fifteen seasons in the National Basketball Association (NBA), winning six NBA championships with the Chicago Bulls. Jordan is the principal owner and chairman of the Charlotte Hornets of the NBA and of 23XI Racing in the NASCAR Cup Series. His biography on the official NBA website states: "By acclamation, Michael Jordan is the greatest basketball player of all time." He was integral in popularizing the NBA around the world in the 1980s and 1990s, becoming a global cultural icon in the process.'
})
assert client.exists(index='my-first-index',id=23)

Now we've created an index and indexed one document - not bad, but we're just getting started! Let's try to search our data.

In [ ]:
client.search(query={
    'match': {
        'message': 'Michael'
    }
})

In [ ]:
client.search(query={
    'match': {
        'message': 'GOAT'
    }
})

# Growing the set

We can already explore the search API with this minimal example, however we would always end up with one of two result sets (one result or none).

To make things more interesting, we should probably accumulate a bit more data. I've prepared some random articles from Wikipedia to ingest into Elasticsearch.

In [ ]:
from collections import namedtuple

import json, os

for _article in os.listdir('./_articles/'):
    with open(f'./_articles/{_article}', 'r', encoding='utf-8') as file:
        _doc = json.load(file)

        client.index(
            index='my-second-index',
            document={
                'url': _doc['url'],
                'title': _doc['title'],
                'summary': _doc['summary'],
                'content': _doc['content'],
                'links': _doc['links'],
            }
        )

assert client.count(index='my-second-index')['count'] == 1000, 'Apparently ingestion of some articles failed ¯\_(ツ)_/¯'